In [1]:
import numpy as np
import pandas as pd
import scipy.stats as si
from itertools import product

In [2]:
np.random.seed(123)

In [3]:
def black_scholes_put_option(S, K, T, q, r, sigma):
    '''
    S: Stock price
    K: Strike price
    T: Maturity
    q: Dividend rate
    r: Risk free rate
    sigma: Volatility
    '''
    
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    put = K * np.exp(-r * T) * si.norm.cdf(-d2) - S * np.exp(-q * T) * si.norm.cdf(-d1)
    
    return put

In [4]:
# 184 * 60 * 32 * number of sigma values = 353280 * x
sim_size = 59 * 20 * 19 * 50

In [5]:
S = np.ones(sim_size) * 100

K_range = np.array(list(np.arange(2, 80, 5)) + 
                   list(np.arange(81, 108)) +
                   list(np.arange(108, 185, 5))
                  )
T_range = np.arange(0.01, 1, 0.05)

q = np.zeros(sim_size)

r = np.array([0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.24, 0.29, 0.89,
       0.95, 1.13, 1.41, 1.46, 1.54, 1.57, 1.6 ]) / 100

sigma_range = np.zeros(50)
for i in range(50):
    sigma_sim = 0 
    while sigma_sim <= 0.086 or sigma_sim >= 2.17:
        sigma_sim = np.random.normal(0.411271, 0.237628, 1)
    sigma_range[i] = sigma_sim

print('Number of different values for T, Strike, r and sigma are {}, {}, {}, {}, respectively.'.format(
    len(T_range), len(K_range), len(r), len(sigma_range)))

Number of different values for T, Strike, r and sigma are 20, 59, 19, 50, respectively.


In [6]:
# cartesian product 
matrix = np.array(list(product(K_range, T_range, r, sigma_range)))
matrix.shape

(1121000, 4)

In [7]:
K = matrix[:,0]
T = matrix[:,1]
r = matrix[:,2]
sigma = matrix[:,3]

In [8]:
put_price = black_scholes_put_option(S, K, T, q, r, sigma)

In [9]:
sim_data = pd.DataFrame(
    np.vstack((S, K, T, q, r, sigma, put_price)).T,
    columns = ['Stock price', 'Strike price', 'Maturity', 
               'Dividend rate', 'Risk free rate', 'Volatility', 'Put Price'] 
)

In [10]:
sim_data.head()

,Stock price,Strike price,Maturity,Dividend rate,Risk free rate,Volatility,Put Price
0,100.0,2.0,0.01,0.0,0.0002,0.153295,0.0
1,100.0,2.0,0.01,0.0,0.0002,0.648268,0.0
2,100.0,2.0,0.01,0.0,0.0002,0.478515,0.0
3,100.0,2.0,0.01,0.0,0.0002,0.273779,0.0
4,100.0,2.0,0.01,0.0,0.0002,0.803699,0.0


In [11]:
sim_data.to_csv("/Users/gaojinglun/Desktop/RSG/ES_EZ_Put_Sim.csv")